# Abstract classes

---

### Example

In [1]:
# library.py

class Base:
    def bar(self):
        return self.baz()  # mast be implemented in derived class

In [2]:
# user.py

class Derived(Base):
    def baz(self):   # baz is not defined
        return 'baz'

In [3]:
d = Derived()
print(d.bar())

baz


## How to enforce a constraint from a base class to a derived class?

(How to make sure that the `baz` methot is implemented in a derived class?)

---

### Everything in Python is dynamic.

(Almost) Everything you write in Python is an executable code.

For example, a class can be created inside a function! Bacause the `class` statement is an actual executable code: 

In [4]:
def f():
    class SomeClass:
        pass

Let's look at the assembly:

In [5]:
from dis import dis
dis(f)

  2           0 LOAD_BUILD_CLASS
              2 LOAD_CONST               1 (<code object SomeClass at 0x7fa134f22c00, file "<ipython-input-4-5eacf1cbe947>", line 2>)
              4 LOAD_CONST               2 ('SomeClass')
              6 MAKE_FUNCTION            0
              8 LOAD_CONST               2 ('SomeClass')
             10 CALL_FUNCTION            2
             12 STORE_FAST               0 (SomeClass)
             14 LOAD_CONST               0 (None)
             16 RETURN_VALUE

Disassembly of <code object SomeClass at 0x7fa134f22c00, file "<ipython-input-4-5eacf1cbe947>", line 2>:
  2           0 LOAD_NAME                0 (__name__)
              2 STORE_NAME               1 (__module__)
              4 LOAD_CONST               0 ('f.<locals>.SomeClass')
              6 STORE_NAME               2 (__qualname__)

  3           8 LOAD_CONST               1 (None)
             10 RETURN_VALUE


There is no reason in Python why we can't do this:

In [6]:
for _ in range(10):
    class SomeClass:
        pass

## 1. Using build_class hook
---

In [7]:
# user.py
import builtins
orig_build_class = __build_class__

def __build_class__(cls, name, bases):
    print('__build_class__', cls, name, bases)
    if not hasattr(cls, 'baz'):
        raise TypeError('aaaaaaa')
    return orig_build_class(cls, name, bases)

# builtins.__build_class__ = __build_class__

class Derived(Base):
    def baz(self):   # baz is not defined
        return 'baz'

# d = Derived()
# d.baz()

# builtins.__build_class__ = orig_build_class

## 2. Using metaclasses
---

But this is not a good way to this. The better way is to use metaclass!

## How do metaclasses work

Every class in Python is an instance of `type` metaclass.

Some class can be created using `type`.

You can call type() with three arguments — `type(<name>, <bases>, <dct>)`:

- <name> specifies the class name. This becomes the `__name__` attribute of the class.
- <bases> specifies a tuple of the base classes from which the class inherits. This becomes the `__bases__` attribute of the class.
- <dct> specifies a namespace dictionary containing definitions for the class body. This becomes the `__dict__` attribute of the class.

In [8]:
MyClass = type('MyClass', (), {})

# the same as
# class MyClass:
#     pass

print(MyClass)
print(f'Type of my class:          {type(MyClass)}')
print(f'Type of my class instance: {type(MyClass())}')

<class '__main__.MyClass'>
Type of my class:          <class 'type'>
Type of my class instance: <class '__main__.MyClass'>


So we can inherit from `type` to make our own metaclasses:

In [9]:
# library.py

class BaseMeta(type):
    def __new__(cls, name, bases, body):
        print('__new__', name, bases, body)
        
        # check if 'baz' is present
        if 'baz' not in body:
            raise TypeError('baz is not present')
        
        return super().__new__(cls, name, bases, body)

class Base(metaclass=BaseMeta):
    def bar(self):
        return self.baz()  # mast be implemented in derived class

__new__ Base () {'__module__': '__main__', '__qualname__': 'Base', 'bar': <function Base.bar at 0x7fa134f459d8>}


TypeError: baz is not present

## 3. ABC
---

https://docs.python.org/3/library/abc.html

In [ ]:
# TODO